# density-缩减版，可以一键计算-2022年7-8月-加了一个惩罚项

要点梳理：
- 剔除云层遮挡和数据缺失的情况

拖山的经度和纬度是(120.14,31.406)，兰山嘴的经度和纬度是(119.89,31.212)。我需要分析这2个点位逐日的叶绿素a浓度，请你修改下面的代码。如果受到云层影响导致数据缺失，请你用NAN代替。

创新点在于，将云层覆盖的区域设置为999999，这样在后续处理中可以很容易地识别出这些区域。

https://engine-aiearth.aliyun.com/docs/page/api?d=07f36f#heading-22

https://engine-aiearth.aliyun.com/?spm=a2cvf.28031243.0.0.5dc743d42pbV5o#/utility/auth-token

In [15]:
import aie

aie.Authenticate(token='8f04bf5ebc3b343fcfc4373fd0dd4a33')
aie.Initialize()

计算资源初始化中，请等待...
计算资源初始化完成.


In [16]:
import geopandas as gpd

In [18]:
# 读取shp文件
shapefile_path = '/root/Download/Modis-algae-aliyun/local-data/taihu.shp'
gdf = gpd.read_file(shapefile_path)

# 假设shp文件中只有一个几何对象，取第一个
geometry = gdf.geometry.iloc[0]
print(geometry)
# 输出((119.95082568376117 31.341888097180494, 119.98639104240964 31.388554715968827, ...

# 将geopandas的geometry对象转换为ee.Geometry
coords = list(geometry.exterior.coords)
flattened_coords = [coord for point in coords for coord in point]
taihu = aie.Geometry.Polygon(flattened_coords)
# 构造Polygon。输入参数支持GeoJSON 'Polygon'类型格式的坐标点集，
# 同时也支持包含偶数个数字的一维List，例如aie.Geometry.Polygon([aLng, aLat, bLng, bLat, ...])。
# 需注意，输入的坐标点将以EPSG:4326坐标系进行解析。

# 打印ee.Geometry对象以验证
print(taihu)

POLYGON ((119.95082568376117 31.341888097180494, 119.98639104240964 31.388554715968827, 120.00611934273043 31.40820067954559, 120.0213482061359 31.420557311454473, 120.02780893606551 31.42562749019076, 120.03023170978913 31.42966374717613, 120.0274051404449 31.43596389857637, 120.02025218945138 31.447873965243872, 120.00888822698595 31.45166321445573, 120.01067646473432 31.455403102339847, 120.01448368058568 31.46224277412878, 120.01806015608244 31.467556706624215, 120.02342486932757 31.473017934057427, 120.02827041677476 31.47729813281918, 120.02902032292732 31.479561147303205, 120.03525031250228 31.481725718642263, 120.03871141782174 31.48452974796941, 120.04223020822982 31.48275879185581, 120.04678733023371 31.479659537995836, 120.05734370145801 31.476461787529658, 120.06017027080222 31.47651098451827, 120.06380443138762 31.471541958062947, 120.06276609979179 31.468442332717288, 120.06345832085567 31.46578542930369, 120.06703479635243 31.4656870240255, 120.07078432711515 31.46052060

# 计算完整的7月份

你这个逻辑还是有问题，请你用伪代码的方式，提供一个代码的逻辑框架，please think by chain-of-thought：


我需要你同步输出modis反演的藻密度和lanshanzui.csv'中的藻密度，使得二者之间的差距尽可能小。由于lanshanzui.csv'中的藻密度一天会产生6个值，因此lanshanzui.csv'中的藻密度将采用置信区间的形式，modis反演的藻密度将与“lanshanzui.csv'中的藻密度将采用置信区间”进行比较

请你分析并修改这段代码，我的顾虑是："""这段代码好像是通过2022年7月逐日的计算值逐步优化参数，应该根据遥感影像的sample的完整的2022年7月的结果一起计算？输出一个最终的优化参数？"""

需要你注意如下2种情况将不作为参数优化的计算依据：
- MODIS遥感影像中藻密度为9999.0，也就是云层遮挡的情况
- df['density']中数据缺失的情况。


## 改进优化函数

1. 增加模型复杂度：我将原来的线性模型改为了一个结合了指数和线性项的模型。新模型为：density = a * exp(b * NDCI) + c * NDCI + d。这样可以更好地捕捉NDCI和密度之间可能存在的非线性关系。

2. 改进优化算法：我用差分进化算法（differential_evolution）替换了Nelder-Mead方法。差分进化是一种全局优化算法，更适合处理这种可能存在多个局部最优解的问题。

3. 添加正则化：在优化函数中添加了一个正则化项，以防止过拟合。

4. 设置参数边界：为每个参数设置了合理的边界，这有助于算法在有意义的参数空间内搜索。

5. 可视化结果：添加了一个简单的绘图函数，可以直观地看到优化后的模型效果。

这些修改应该能够改善优化过程，使模型更准确，更容易收敛。如果你运行这个修改后的代码，应该会得到更好的结果。

如果这个版本仍然不能很好地解决问题，我们可以考虑以下进一步的改进：

1. 数据预处理：检查并处理异常值，可能的话对数据进行归一化。
2. 特征工程：考虑添加更多的特征，比如NDCI的平方项，或者其他相关的遥感指数。
3. 交叉验证：实施k-fold交叉验证，以确保模型的稳定性和泛化能力。
4. 尝试其他模型：如果这个模型仍然不能很好地拟合数据，可以考虑使用更复杂的模型，如多项式回归或机器学习模型（如随机森林或梯度提升树）。

你可以运行这个新版本的代码，看看结果如何。如果还有任何问题或需要进一步的调整，请告诉我。

## 优化算法

1. 错误信息表明在使用并行处理时出现了问题。可能是 differential_evolution 函数在并行执行时遇到了兼容性问题。

2. 建议尝试以下修改:

   a. 将 workers 参数设置为 1,禁用并行处理:
    ```
      optimization_result = differential_evolution(
          optimize_function, 
          bounds, 
          args=(valid_ndci_list, valid_upper_density_list, valid_lower_density_list),
          maxiter=1000,
          popsize=20,
          mutation=(0.5, 1),
          recombination=0.7,
          updating='deferred',
          workers=1  # 设置为1禁用并行处理
      )

    ```
   b. 如果仍然出错,可以尝试使用其他优化算法,如 Nelder-Mead:
    ```
   
      from scipy.optimize import minimize
      
      optimization_result = minimize(
          optimize_function, 
          initial_params,
          args=(valid_ndci_list, valid_upper_density_list, valid_lower_density_list),
          method='Nelder-Mead',
          options={'maxiter': 10000, 'maxfev': 15000}
      )
    ```

3. 如果问题仍然存在,建议检查 optimize_function 的实现,确保它能正确处理输入参数并返回预期的结果。

4. 另外,可以尝试减少数据量或简化模型,看看是否能够成功运行。如果可以,再逐步增加复杂度。

5. 最后,确保所有必要的库都已正确安装且版本兼容。

请根据这些建议修改代码并重新运行,看看是否能解决问题。如果问题仍然存在,可能需要进一步调查 differential_evolution 函数的使用方法或考虑使用其他优化算法。

## 正则项的优化

让我们来分析一下这个优化函数中正则化项的设计：



在给定的代码中，优化函数定义如下：

```python
def optimize_function(params, ndci_list, upper_density_list, lower_density_list):
    a, b, c, d = params  # 增加参数数量
    predicted_density = a * np.exp(b * np.array(ndci_list)) + c * np.array(ndci_list) + d
    error = np.sum(np.maximum(0, np.array(lower_density_list) - predicted_density, predicted_density - np.array(upper_density_list))**2)
    
    # 添加正则化项
    regularization = 0.01 * (a**2 + b**2 + c**2 + d**2)
    
    return error + regularization
```

正则化项的设计是：

```python
regularization = 0.01 * (a**2 + b**2 + c**2 + d**2)
```

分析这个正则化项的设计：

1. 形式：这是一个L2正则化（也称为岭回归或权重衰减）的变体。它对所有参数的平方和进行惩罚。

2. 强度：正则化强度为0.01，这是一个相对较小的值，意味着正则化的影响相对较轻。

3. 均等处理：所有参数（a, b, c, d）被同等对待，没有考虑它们可能有不同的尺度或重要性。

4. 目的：这种正则化可以帮助防止过拟合，使模型参数保持在较小的范围内。

考虑到这些因素，这个正则化项的设计有以下几点可以改进：

1. 参数尺度：如果参数a, b, c, d的尺度差异很大，可能需要对它们进行不同程度的正则化。例如，可以为每个参数设置不同的正则化系数。

2. 正则化强度：0.01可能适合某些情况，但最好通过交叉验证来确定最佳的正则化强度。

3. 非线性项：考虑到模型中包含指数项，可能需要对非线性项（如exp(b)）进行特殊处理。

4. 正则化类型：除了L2正则化，还可以考虑L1正则化（Lasso）或弹性网络（Elastic Net），它们可能更适合某些特定情况。

改进建议：

1. 引入可调节的正则化强度：
   ```python
   lambda_reg = 0.01  # 可以通过交叉验证调整这个值
   regularization = lambda_reg * (a**2 + b**2 + c**2 + d**2)
   ```

2. 为不同参数设置不同的正则化强度：
   ```python
   regularization = lambda_a * a**2 + lambda_b * b**2 + lambda_c * c**2 + lambda_d * d**2
   ```

3. 考虑模型的非线性性质，可能需要对指数项进行特殊处理：
   ```python
   regularization = lambda_a * a**2 + lambda_b * np.exp(b)**2 + lambda_c * c**2 + lambda_d * d**2
   ```

4. 如果需要更强的稀疏性，可以考虑添加L1正则化：
   ```python
   regularization = lambda_l2 * (a**2 + b**2 + c**2 + d**2) + lambda_l1 * (abs(a) + abs(b) + abs(c) + abs(d))
   ```

总的来说，虽然当前的正则化设计是一个好的开始，但还有改进的空间。最佳的正则化方法通常需要根据具体问题和数据特征来确定，可能需要进行一些实验和调整。

## demo-1.0版本-计算失败

In [ ]:
import datetime
import csv
import aie
import pandas as pd
import numpy as np
import chardet
from datetime import timedelta
from scipy.optimize import minimize, differential_evolution

# 步骤1: 数据准备
csv_file = '/root/Download/Modis-algae-aliyun/aie-taihu/00-lanshanzui.csv'

# 检测文件编码
with open(csv_file, 'rb') as f:
    result = chardet.detect(f.read())

# 使用探测到的编码读取文件
df = pd.read_csv(csv_file, encoding=result['encoding'])

# 将日期列转换为日期时间格式
df['date'] = pd.to_datetime(df['date'])

# 设置日期为索引
df.set_index('date', inplace=True)

# 移除密度为NaN的数据
df = df.dropna(subset=['density'])
# 按日期分组并计算每日的统计数据
daily_stats = df['density'].resample('D').agg(['mean', 'std', 'count'])

# 计算95%置信区间
confidence_interval = 1.96 * daily_stats['std'] / np.sqrt(daily_stats['count'])

# 计算置信区间的上下限
daily_stats['lower_ci'] = daily_stats['mean'] - confidence_interval
daily_stats['upper_ci'] = daily_stats['mean'] + confidence_interval

# 步骤2: 定义优化函数
def optimize_function(params, ndci_list, upper_density_list, lower_density_list):
    a, b, c, d = params  # 增加参数数量
    predicted_density = a * np.exp(b * np.array(ndci_list)) + c * np.array(ndci_list) + d
    error = np.sum(np.maximum(0, np.array(lower_density_list) - predicted_density, predicted_density - np.array(upper_density_list))**2)
    
    # 添加正则化项
    lambda_l2 = 0.01  # L2正则化系数
    lambda_l1 = 0.005  # L1正则化系数
    regularization = lambda_l2 * (a**2 + b**2 + c**2 + d**2) + lambda_l1 * (abs(a) + abs(b) + abs(c) + abs(d))
    
    # 打印当前参数和误差，以帮助调试
    print(f"当前参数: {params}")
    print(f"误差: {error}")
    print(f"正则化项: {regularization}")
    print(f"总损失: {error + regularization}")
    
    return error + regularization

# 步骤3: 收集所有有效数据
valid_ndci_list = []
valid_upper_density_list = []
valid_lower_density_list = []
start_date = datetime.date(2022, 7, 1)
end_date = datetime.date(2022, 8, 31)
delta = datetime.timedelta(days=1)
roi = aie.Geometry.BBox(119.89, 31.212, 119.90, 31.222)

def mask_clouds(image):
    qa = image.select('state_1km')
    cloud_bit_mask = 1 << 10
    mask = qa.bitwiseAnd(aie.Image.constant(cloud_bit_mask))
    return mask

while start_date <= end_date:
    date_str = start_date.strftime("%Y_%m_%d")
    img = aie.Image(f'MODIS_MOD09GA_061_{date_str}')\
             .select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03','state_1km'])
    
    cloud_mask = mask_clouds(img)
    ndci = img.normalizedDifference(['sur_refl_b04', 'sur_refl_b03'])
    ndci_masked = ndci.where(cloud_mask.Not(), aie.Image.constant(9999))
    
    current_date = pd.to_datetime(start_date)
    if current_date in daily_stats.index:
        lower_ci = daily_stats.loc[current_date, 'lower_ci']
        upper_ci = daily_stats.loc[current_date, 'upper_ci']
        
        ndci_values = ndci_masked.sample(roi, 100, 1, 0, True).aggregate_array('nd').getInfo()
        
        valid_ndci = [value for value in ndci_values if value != 9999.0]
        if len(valid_ndci) > 0:
            valid_ndci_list.extend(valid_ndci)
            valid_upper_density_list.extend([upper_ci] * len(valid_ndci))
            valid_lower_density_list.extend([lower_ci] * len(valid_ndci))
    
    start_date += delta

# 打印收集到的数据统计信息
print(f"有效NDCI数量: {len(valid_ndci_list)}")
print(f"NDCI范围: {min(valid_ndci_list)} 到 {max(valid_ndci_list)}")
print(f"密度上限范围: {min(valid_upper_density_list)} 到 {max(valid_upper_density_list)}")
print(f"密度下限范围: {min(valid_lower_density_list)} 到 {max(valid_lower_density_list)}")

# 步骤4: 优化参数
initial_params = [1000, 0.1, 100, 0]  # 增加初始参数数量
bounds = [(0, 10000), (-10, 10), (-1000, 1000), (-1000, 1000)]  # 设置参数边界

optimization_result = minimize(
    optimize_function, 
    initial_params,
    args=(valid_ndci_list, valid_upper_density_list, valid_lower_density_list),
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 1000, 'disp': True}
)

optimal_params = optimization_result.x

# 输出优化结果
print("优化结果:")
print(f"参数: {optimal_params}")
print(f"函数值: {optimization_result.fun}")
print(f"迭代次数: {optimization_result.nit}")
print(f"函数评估次数: {optimization_result.nfev}")
print(f"优化是否成功: {optimization_result.success}")
print(f"优化状态: {optimization_result.message}")


## try其他的参数

In [19]:
import datetime
import csv
import aie
import pandas as pd
import numpy as np
import chardet
from datetime import timedelta
from scipy.optimize import minimize, differential_evolution

# 步骤1: 数据准备
csv_file = '/root/Download/Modis-algae-aliyun/aie-taihu/00-lanshanzui.csv'

# 检测文件编码
with open(csv_file, 'rb') as f:
    result = chardet.detect(f.read())

# 使用探测到的编码读取文件
df = pd.read_csv(csv_file, encoding=result['encoding'])

# 将日期列转换为日期时间格式
df['date'] = pd.to_datetime(df['date'])

# 设置日期为索引
df.set_index('date', inplace=True)

# 移除密度为NaN的数据
df = df.dropna(subset=['density'])
# 按日期分组并计算每日的统计数据
daily_stats = df['density'].resample('D').agg(['mean', 'std', 'count'])

# 计算95%置信区间
confidence_interval = 1.96 * daily_stats['std'] / np.sqrt(daily_stats['count'])

# 计算置信区间的上下限
daily_stats['lower_ci'] = daily_stats['mean'] - confidence_interval
daily_stats['upper_ci'] = daily_stats['mean'] + confidence_interval

# 步骤2: 定义优化函数
def optimize_function(params, ndci_list, upper_density_list, lower_density_list):
    a, b, c, d = params
    predicted_density = a * np.exp(b * np.array(ndci_list)) + c * np.array(ndci_list) + d
    error = np.sum(np.maximum(0, np.array(lower_density_list) - predicted_density, predicted_density - np.array(upper_density_list))**2)
    
    lambda_l2 = 0.01
    lambda_l1 = 0.005
    regularization = lambda_l2 * (a**2 + b**2 + c**2 + d**2) + lambda_l1 * (abs(a) + abs(b) + abs(c) + abs(d))
    
    total_loss = error + regularization
    
    print(f"当前参数: {params}")
    print(f"误差: {error}")
    print(f"正则化项: {regularization}")
    print(f"总损失: {total_loss}")
    
    return total_loss

# 步骤3: 收集所有有效数据
valid_ndci_list = []
valid_upper_density_list = []
valid_lower_density_list = []
start_date = datetime.date(2022, 7, 1)
end_date = datetime.date(2022, 8, 31)
delta = datetime.timedelta(days=1)
roi = aie.Geometry.BBox(119.89, 31.212, 119.90, 31.222)

def mask_clouds(image):
    qa = image.select('state_1km')
    cloud_bit_mask = 1 << 10
    mask = qa.bitwiseAnd(aie.Image.constant(cloud_bit_mask))
    return mask

while start_date <= end_date:
    date_str = start_date.strftime("%Y_%m_%d")
    img = aie.Image(f'MODIS_MOD09GA_061_{date_str}')\
             .select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03','state_1km'])
    
    cloud_mask = mask_clouds(img)
    ndci = img.normalizedDifference(['sur_refl_b04', 'sur_refl_b03'])
    ndci_masked = ndci.where(cloud_mask.Not(), aie.Image.constant(9999))
    
    current_date = pd.to_datetime(start_date)
    if current_date in daily_stats.index:
        lower_ci = daily_stats.loc[current_date, 'lower_ci']
        upper_ci = daily_stats.loc[current_date, 'upper_ci']
        
        ndci_values = ndci_masked.sample(roi, 100, 1, 0, True).aggregate_array('nd').getInfo()
        
        valid_ndci = [value for value in ndci_values if value != 9999.0]
        if len(valid_ndci) > 0:
            valid_ndci_list.extend(valid_ndci)
            valid_upper_density_list.extend([upper_ci] * len(valid_ndci))
            valid_lower_density_list.extend([lower_ci] * len(valid_ndci))
    
    start_date += delta

print(f"有效NDCI数量: {len(valid_ndci_list)}")
print(f"NDCI范围: {min(valid_ndci_list)} 到 {max(valid_ndci_list)}")
print(f"密度上限范围: {min(valid_upper_density_list)} 到 {max(valid_upper_density_list)}")
print(f"密度下限范围: {min(valid_lower_density_list)} 到 {max(valid_lower_density_list)}")

# 步骤4: 优化参数
initial_params = [1000, 0.1, 100, 0]
bounds = [(0, 10000), (-10, 10), (-1000, 1000), (-1000, 1000)]

optimization_result = minimize(
    optimize_function, 
    initial_params,
    args=(valid_ndci_list, valid_upper_density_list, valid_lower_density_list),
    method='L-BFGS-B',
    bounds=bounds,
    options={'maxiter': 1000, 'disp': True}
)

optimal_params = optimization_result.x

print("优化结果:")
print(f"参数: {optimal_params}")
print(f"函数值: {optimization_result.fun}")
print(f"迭代次数: {optimization_result.nit}")
print(f"函数评估次数: {optimization_result.nfev}")
print(f"优化是否成功: {optimization_result.success}")
print(f"优化状态: {optimization_result.message}")

# 添加数据验证和错误处理
if not optimization_result.success:
    print("优化失败，正在进行问题诊断...")
    
    # 检查数据
    print(f"NDCI值的范围: {min(valid_ndci_list)} 到 {max(valid_ndci_list)}")
    print(f"密度上限的范围: {min(valid_upper_density_list)} 到 {max(valid_upper_density_list)}")
    print(f"密度下限的范围: {min(valid_lower_density_list)} 到 {max(valid_lower_density_list)}")
    
    # 检查是否有异常值
    ndci_outliers = [x for x in valid_ndci_list if x < -1 or x > 1]
    density_outliers = [x for x in valid_upper_density_list + valid_lower_density_list if x < 0 or x > 1e9]
    
    if ndci_outliers:
        print(f"发现 {len(ndci_outliers)} 个NDCI异常值")
    if density_outliers:
        print(f"发现 {len(density_outliers)} 个密度异常值")
    
    # 尝试不同的初始参数
    print("尝试不同的初始参数...")
    for _ in range(5):
        new_initial_params = np.random.uniform(low=[0, -10, -1000, -1000], high=[10000, 10, 1000, 1000])
        new_result = minimize(
            optimize_function, 
            new_initial_params,
            args=(valid_ndci_list, valid_upper_density_list, valid_lower_density_list),
            method='L-BFGS-B',
            bounds=bounds,
            options={'maxiter': 1000, 'disp': True}
        )
        if new_result.success:
            print("找到新的有效解:")
            print(f"参数: {new_result.x}")
            print(f"函数值: {new_result.fun}")
            break
    else:
        print("尝试新的初始参数仍然失败")
    
    # 尝试不同的优化算法
    print("尝试使用差分进化算法...")
    de_result = differential_evolution(
        optimize_function,
        bounds,
        args=(valid_ndci_list, valid_upper_density_list, valid_lower_density_list),
        maxiter=100,
        popsize=20,
        disp=True
    )
    if de_result.success:
        print("差分进化算法找到解:")
        print(f"参数: {de_result.x}")
        print(f"函数值: {de_result.fun}")
    else:
        print("差分进化算法也失败了")

    print("建议: 检查数据质量，考虑数据预处理，或者重新设计目标函数")


有效NDCI数量: 46
NDCI范围: -0.052804604172706604 到 0.20655737817287445
密度上限范围: 14983947.220276179 到 119769899.46780816
密度下限范围: -593918.0570126772 到 63902812.23538995
当前参数: [1.e+03 1.e-01 1.e+02 0.e+00]
误差: nan
正则化项: 10105.500600000001
总损失: nan
当前参数: [1.e+03 1.e-01 1.e+02 0.e+00]
误差: nan
正则化项: 10105.50060020005
总损失: nan
当前参数: [1.0000000e+03 1.0000001e-01 1.0000000e+02 0.0000000e+00]
误差: nan
正则化项: 10105.500600000069
总损失: nan
当前参数: [1.e+03 1.e-01 1.e+02 0.e+00]
误差: nan
正则化项: 10105.50060002005
总损失: nan
当前参数: [1.e+03 1.e-01 1.e+02 1.e-08]
误差: nan
正则化项: 10105.50060000005
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan nan nan]
误差: nan
正则化项: nan
总损失: nan
当前参数: [nan nan na


 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.

 Line search cannot locate an adequate point after MAXLS
  function and gradient evaluations.
  Previous x, f and g restored.
 Possible causes: 1 error in function or gradient evaluation;
                  2 rounding error dominate computation.

 Line search ca

当前参数: [6452.6923339     6.64231878 -733.48951021   70.7941932 ]
误差: nan
正则化项: 421839.32964959415
总损失: nan
当前参数: [ 4.84560141e+03 -1.93107709e+00  4.67140001e+02 -9.88989849e+02]
误差: nan
正则化项: 246793.2928377855
总损失: nan
当前参数: [4287.75690221    8.50206207 -281.55532258  671.44161202]
误差: nan
正则化项: 189176.6340349109
总损失: nan
当前参数: [ 8.20784224e+03 -8.02198071e+00  1.25458785e+02  1.06584628e+02]
误差: nan
正则化项: 674000.6273317755
总损失: nan
当前参数: [ 9.58890537e+03 -5.61492619e+00  7.68539297e+02  6.80993897e+02]
误差: nan
正则化项: 930070.6500467323
总损失: nan
当前参数: [ 4.51683379e+03 -2.29720997e+00 -2.14897521e+02 -9.19619532e+02]
误差: nan
正则化项: 212965.00652889325
总损失: nan
当前参数: [ 2.94018350e+03 -6.57292545e-01  4.02768024e+02  1.77241453e+01]
误差: nan
正则化项: 88088.9631848125
总损失: nan
当前参数: [ 6.49452770e+03  1.76901955e+00 -3.99372472e+02 -7.59020594e+02]
误差: nan
正则化项: 429183.31145378586
总损失: nan
当前参数: [1632.35681591   -6.88872315 -582.35910622  285.66833724]
误差: nan
正则化项: 30866.38392946735
总损失: nan
当前参数:

In [10]:
import matplotlib.pyplot as plt
import numpy as np
import datetime
from datetime import timedelta

# 步骤5: 应用优化后的参数并绘制结果
start_date = datetime.date(2022, 7, 1)
end_date = datetime.date(2022, 8, 31)
dates = []
estimated_densities = []
lower_ci_values = []
upper_ci_values = []

def estimate_density(ndci, params):
    a, b, c, d = params
    return a * np.exp(b * ndci) + c * ndci + d

while start_date <= end_date:
    date_str = start_date.strftime("%Y_%m_%d")
    img = aie.Image(f'MODIS_MOD09GA_061_{date_str}')\
             .select(['sur_refl_b01', 'sur_refl_b04', 'sur_refl_b03','state_1km'])
    
    cloud_mask = mask_clouds(img)
    ndci = img.normalizedDifference(['sur_refl_b04', 'sur_refl_b03'])
    ndci_masked = ndci.where(cloud_mask.Not(), aie.Image.constant(9999))
    
    # 计算估算结果
    ndci_values = ndci_masked.sample(roi, 100, 1, 0, True).aggregate_array('nd').getInfo()
    
    # 过滤掉云覆盖的像素
    valid_ndci = [value for value in ndci_values if value != 9999.0]
    
    # 输出过滤前后的像素数量，以便检查
    print(f"过滤前像素数量: {len(ndci_values)}")
    print(f"过滤后像素数量: {len(valid_ndci)}")
    
    # 估算藻密度
    if valid_ndci:
        estimated_density = [estimate_density(ndci, optimal_params) for ndci in valid_ndci]
    else:
        estimated_density = []
    
    # 获取置信区间
    current_date = pd.to_datetime(start_date)
    if current_date in daily_stats.index:
        lower_ci = daily_stats.loc[current_date, 'lower_ci']
        upper_ci = daily_stats.loc[current_date, 'upper_ci']
    else:
        lower_ci = upper_ci = None
    
    # 存储结果
    dates.append(start_date)
    estimated_densities.append(np.mean(estimated_density) if estimated_density else np.nan)
    lower_ci_values.append(lower_ci)
    upper_ci_values.append(upper_ci)
    
    print(f"日期: {date_str}")
    print(f"藻密度估算结果: {np.mean(estimated_density) if estimated_density else '数据不可用'}")
    print(f"置信区间: [{lower_ci}, {upper_ci}]")
    print("---")
    
    start_date += timedelta(days=1)

# 绘制结果
plt.figure(figsize=(12, 6))
plt.plot(dates, estimated_densities, label='估算结果', color='blue', marker='o')
plt.fill_between(dates, lower_ci_values, upper_ci_values, alpha=0.2, color='blue', label='95%置信区间')

plt.xlabel('日期')
plt.ylabel('藻密度')
plt.title('2022年7-8月太湖藻密度估算结果')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()

# 添加云覆盖说明
plt.text(0.95, 0.05, '缺失数据表示云覆盖', transform=plt.gca().transAxes, 
         horizontalalignment='right', verticalalignment='bottom', color='red')

plt.show()

# 步骤6: 输出最终结果
print("最优参数:", optimal_params)
print("整月藻密度估算已完成")

过滤前像素数量: 1
过滤后像素数量: 1
日期: 2022_07_01
藻密度估算结果: 1004.7031944033115
置信区间: [15381023.755087659, 21552309.578245677]
---
过滤前像素数量: 1
过滤后像素数量: 0
日期: 2022_07_02
藻密度估算结果: 数据不可用
置信区间: [11330271.824073968, 16669728.175926032]
---
过滤前像素数量: 1
过滤后像素数量: 1
日期: 2022_07_03
藻密度估算结果: 1006.8721881742052
置信区间: [11649386.113057153, 14983947.220276179]
---
过滤前像素数量: 1
过滤后像素数量: 1
日期: 2022_07_04
藻密度估算结果: 1029.4258218414448
置信区间: [15691375.81649267, 26008624.18350733]
---
过滤前像素数量: 1
过滤后像素数量: 1
日期: 2022_07_05
藻密度估算结果: 1009.9636186099402
置信区间: [12101820.94093789, 19731512.392395444]
---
过滤前像素数量: 1
过滤后像素数量: 0
日期: 2022_07_06
藻密度估算结果: 数据不可用
置信区间: [12971030.604999147, 27053512.72833419]
---
过滤前像素数量: 1
过滤后像素数量: 0
日期: 2022_07_07
藻密度估算结果: 数据不可用
置信区间: [7966451.605437659, 47594988.39456234]
---
过滤前像素数量: 1
过滤后像素数量: 1
日期: 2022_07_08
藻密度估算结果: 1013.0291086908221
置信区间: [19055410.563993074, 28277922.76934026]
---
过滤前像素数量: 1
过滤后像素数量: 1
日期: 2022_07_09
藻密度估算结果: 1009.0543574175223
置信区间: [18796537.695188735, 39403462.30481127]
---
过滤前

KeyboardInterrupt: 